In [ ]:
import numpy as np
import math
import random
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
#Upload dataset
from google.colab import files
uploaded = files.upload()

In [ ]:
#Read dataset
import io
ttt = pd.read_csv(io.BytesIO(uploaded['tic-tac-toe.csv']))
ttt = ttt.replace({
    #Covert output values to numerical values from categorical values
    "Class":{
        "negative":0,
        "positive":1
    }
})
#Spliting the features and the label
ttt_x = ttt.loc[:, ttt.columns != "Class"]
ttt_y = ttt.loc[:, "Class"]

In [ ]:
#Split dataset into 80% traning data and 20% testing data
train_data, test_data = train_test_split(ttt, train_size=0.8, stratify = ttt_y, random_state=1997)

In [ ]:
class Node:
    def __init__(self, attribute=None, attribute_values=None, child_nodes=None, decision=None):
        self.attribute = attribute
        self.attribute_values = attribute_values
        self.child_nodes = child_nodes
        self.decision = decision

In [ ]:
class DecisionTree:

    root = None

    @staticmethod
    def plurality_values(data):
        labels = data[:, data.shape[1] - 1]  # store the last column in labels
        labels = list(labels)
        zero = labels.count(0)  # Count the occurence of '0'
        one = labels.count(1)  # Count the occurence of '1'
        if zero > one:  # Find who occurs more "0/1", return 0 if 0 occurs more than 1 and vice versa
            return 0
        return 1


    @staticmethod
    def all_zero(data):
        labels = data[:, data.shape[1] - 1]  # store the last column in labels
        labels = list(labels)
        if labels.count(0) == len(labels):  # Check if all the input labels are 0 or not
            return True
        return False

    @staticmethod
    def all_one(data):
        labels = data[:, data.shape[1] - 1]  # store the last column in labels
        labels = list(labels)
        if labels.count(1) == len(labels):  # Check if all the input labels are 0 or not
            return True
        return False

    @staticmethod
    def importance(data, attributes):
        labels = data[:, data.shape[1] - 1]  # store the last column in labels
        labels = list(labels)
        # Here n is equal to q of the entropy formula [[[B(q) = −q × log2q − (1 − q) × log2(1 − q)]]]
        n = labels.count(0)/len(labels)  # Calculating the occurence of 0(n) inside total total labels
        # Here p is equal to (1 - q) of the entropy formula [[[B(q) = −q × log2q − (1 − q) × log2(1 − q)]]]
        p = labels.count(1)/len(labels)  # Calculating the occurence of 1(p) inside total total labels
        l_e = -n*math.log(n, 2) - p*math.log(p, 2)  # Calculating the label entropy where logarithmic base is 2
        gain = {}  # Create a blank gain dictonary to store the gain value
        length = len(attributes)  # Calculate the length of each attribute
        for a in attributes:  # Iterate the attributes elements one by one
            x = data[:, data.shape[1] - length - 1]  #Separate each column based on the current length value
            length -= 1  #Decrement the length value by 1 over every iteration
            uv = np.unique(x)  # Findout all unique values in each column
            x = list(x)  # Convert each column into a list
            pair = list(zip(x, labels))  # Zip each column with list and convert it into a list
            after_split = 0  # Initializing after_split with value 0
            for i in uv:  # Iterate through uv by each unique value
                y = (i, 1)  # Make a touple with the value of unique value and 1
                n = (i, 0)  # Make a touple with the value of unique value and 0
                yes = pair.count(y)  # Count the occurence of unique value and 1 in the pair list
                no = pair.count(n)  # Count the occurence of unique value and 0 in the pair list
                if yes == 0 or no == 0:  # If yes or no is equal to 0 thats mean the entropy is 0
                    entropy = 0
                else:
                    p_yes = yes/(yes + no)  # Calculate the value of q = (p_yes) for the entropy formula
                    p_no = no/(yes + no)  # Calculate the value of (1 - q) = (p_yes) for the entropy formula
                    entropy = -p_yes*math.log(p_yes, 2) - p_no*math.log(p_no, 2)  # Calculate the entropy
                weight = (yes + no)/len(labels)  # Calculate the weight value based on the summation of yes and no
                after_split += weight*entropy  # Update the after_split value for next iteration
            gain_value = l_e - after_split  # Calculate the gain_value
            if a not in gain:  # If the attribute element absent in gain dictonary then add the value in that dictonary and the key is the element of the attribute
                gain[a] = gain_value
        max = -1000  # Assume that the max value of gain is negative 1000
        atr = -1  # Assume that the atr value of gain is negative 1
        for value in gain:  # Iterate gain dictonary by value
            if gain[value] > max:  # Compare if the real gain value is greater than max
                max = gain[value]  # Upadate the max value with real gain value
                atr = value  # Upadate the atr value with real value from gain dictonary
        return atr  # Return the final atr value


    def train(self, data, attributes, parent_data):
        data = np.array(data)
        parent_data = np.array(parent_data)
        attributes = list(attributes)

        if data.shape[0] == 0:  # if x is empty
            return Node(decision=self.plurality_values(parent_data))

        elif self.all_zero(data):
            return Node(decision=0)

        elif self.all_one(data):
            return Node(decision=1)

        elif len(attributes) == 0:
            return Node(decision=self.plurality_values(data))

        else:
            a = self.importance(data, attributes)
            tree = Node(attribute=a, attribute_values=np.unique(data[:, a]), child_nodes=[])
            attributes.remove(a)
            for vk in np.unique(data[:, a]):
                new_data = data[data[:, a] == vk, :]
                subtree = self.train(new_data, attributes, data)
                tree.child_nodes.append(subtree)

            return tree

    def fit(self, data):
        self.root = self.train(data, list(range(data.shape[1] - 1)), np.array([]))

    def predict(self, data):
        predictions = []
        for i in range(data.shape[0]):
            current_node = self.root
            while True:
                if current_node.decision is None:
                    current_attribute = current_node.attribute
                    current_attribute_value = data[i, current_attribute]
                    if current_attribute_value not in current_node.attribute_values:
                        predictions.append(random.randint(0, 1))
                        break
                    idx = list(current_node.attribute_values).index(current_attribute_value)

                    current_node = current_node.child_nodes[idx]
                else:
                    predictions.append(current_node.decision)
                    break

        return predictions

In [ ]:
dt = DecisionTree()
dt.fit(np.array(train_data))
predict = dt.predict(np.array(test_data))

In [ ]:
test_data = np.array(test_data)
test_y = list(test_data[:, test_data.shape[1] - 1])
correct = 0
i = 0
for a in predict:
    if a == test_y[i]:
        correct += 1
    i += 1
accuracy = (correct/len(test_y))*100
tp = (1, 1)  # True positive
tn = (0, 0)  # True negative
fp = (1, 0)  # False positive
fn = (0, 1)  # False negative
mer = list(zip(predict, test_y))
true_positive = mer.count(tp)
true_negative = mer.count(tn)
false_positive = mer.count(fp)
false_negative = mer.count(fn)
precission = true_positive/(true_positive + false_positive)
recall = true_positive/(true_positive + false_negative)
f1_score = (2*precission*recall)/(precission + recall)
print("The accuracy of this algorithm is: {}%".format(accuracy))
print("The precision value of this algorithm is: {}".format(precission))
print("The recall value of this algorithm is: {}".format(recall))
print("F1 score of this algorithm is: {}".format(f1_score))